# Data preparation: Embeddings and Indexing
Este notebook refere-se à etapa de preparação dos dados, contemplando todo o pipeline de vetorização textual. Os passos aqui apresentados são:
- Chunking dos documentos
- Vetorização (embeddings)
- Indexação e armazenamento vetorial
Ao final, os dados de indexação são persistidos para um arquivo local.


## Instalações e importações

In [ ]:
# Instalações
!pip install "datasets<4.0.0"

In [ ]:
# Imports
# Manipulação dos dados
from datasets import load_dataset, load_from_disk

# Embeddings
from transformers import AutoTokenizer

## Carregamento dos dados

In [ ]:
# Load Quati dataset
quati_ds = load_dataset("parquet", data_files="./quati_processed.parquet")

print(quati_ds)
print("-" * 100)
quati_ds[:5]

## Chunking

In [ ]:
# Initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    "sentence-transformers/paraphrase-multilingual-mpnet-base-v2",
    device="cuda"
)

In [ ]:
def chunk_text(text, max_tokens=256, overlap=64):
    tokens = tokenizer.encode(
        text,
        add_special_tokens=False
    )

    chunks = []
    for i in range(0, len(tokens), max_tokens - overlap):
        chunk = tokens[i:i + max_tokens]
        chunks.append(tokenizer.decode(chunk))
    return chunks


def chunk_by_tokens(text, chunk_size=384, overlap=64):
    tokens = tokenizer(
        text,
        return_offsets_mapping=True,
        add_special_tokens=False,
    )
    input_ids = tokens["input_ids"]
    offsets = tokens["offset_mapping"]

    chunks = []
    start = 0

    while start < len(input_ids):
        end = start + chunk_size
        chunk_offsets = offsets[start:end]
        char_start = chunk_offsets[0][0]
        char_end = chunk_offsets[-1][1]
        chunks.append(text[char_start:char_end])
        start += chunk_size - overlap

    return chunks

In [ ]:
# Applying chunks into documents
def apply_chunks(batch):
    batch["chunks"] = [chunk_by_tokens(doc) for doc in batch["passage"]]
    return batch

quati_ds = quati_ds.map(
    apply_chunks,
    batched=True
    batch_size=10_000,
    num_proc=cpu_count
)

quati_ds.select_columns(["chunks", "passage"])[:2]

### Demonstração das funções do Tokenizer

Vamos usar a frase "Olá, mundo! Como vai?" para demonstrar as diferenças entre as funções do `tokenizer`.

In [ ]:
sample_text = "Olá, mundo! Como vai?"

print(f"Texto original: {sample_text}\n")

# 1. tokenizer("texto") - Chamada direta
print("--- tokenizer(\"texto\") ---")
encoded_full = tokenizer(sample_text, return_tensors='pt')
print("Resultado (tensor):", encoded_full)
print("Input IDs (list):", encoded_full['input_ids'].tolist())
print("Attention Mask (list):", encoded_full['attention_mask'].tolist())
print("Decodificado de volta (para referência):", tokenizer.decode(encoded_full['input_ids'][0]))
print("\n")

# 2. tokenizer.tokenize("texto")
print("--- tokenizer.tokenize(\"texto\") ---")
tokenized_strings = tokenizer.tokenize(sample_text)
print("Resultado (tokens em string):", tokenized_strings)
print("\n")

# 3. tokenizer.encode("texto")
print("--- tokenizer.encode(\"texto\") ---")
encoded_ids = tokenizer.encode(sample_text, add_special_tokens=True)
print("Resultado (IDs numéricos):", encoded_ids)
print("Decodificado de volta (para referência):", tokenizer.decode(encoded_ids))
print("\n")

# Exemplo de tokenizer.encode com add_special_tokens=False (como na sua função chunk_text)
print("--- tokenizer.encode(\"texto\", add_special_tokens=False) ---")
encoded_ids_no_special = tokenizer.encode(sample_text, add_special_tokens=False)
print("Resultado (IDs numéricos sem tokens especiais):", encoded_ids_no_special)
print("Decodificado de volta (para referência):", tokenizer.decode(encoded_ids_no_special))

## Vetorização dos dados (embeddings)

## Indexação

## Persistência dos dados